In [1]:
import pandas as pd
import requests
import json
import datetime
import csv
import time
import matplotlib.pyplot as plt
import ast
from dotenv import load_dotenv
import os 
from sqlalchemy import create_engine

In [2]:
load_dotenv()

True

In [3]:
DB_USER = os.getenv("DB_USER")
DB_PASSWORD = os.getenv("DB_PASSWORD")
DB_HOST = os.getenv("DB_HOST")
DB_PORT = os.getenv("DB_PORT")
DB_NAME = os.getenv("DB_NAME")

In [5]:
TOKEN = os.getenv("NYC_open_data_token")

base_url = "https://data.cityofnewyork.us/resource/c3uy-2p5r.json"

headers = {"X-App-Token": TOKEN}

limit = 1000    
offset = 0
all_records = []

while True:
    url = f"{base_url}?$limit={limit}&$offset={offset}"
    try:
        response = requests.get(url, headers=headers, timeout=20)
        response.raise_for_status()
    except requests.exceptions.RequestException as e:
        print("Request failed, retrying in 2 seconds...", e)
        time.sleep(2)
        continue

    batch = response.json()

    if not batch:
        print("No more data returned. Stopping.")
        break

    all_records.extend(batch)
    print(f"Fetched {len(batch)} rows (offset={offset})")

    # Stop if fewer than the limit means end of dataset
    if len(batch) < limit:
        break

    offset += limit
    time.sleep(0.2)  # polite rate-limit protection


# dataframe:

air_quality = pd.DataFrame(all_records)

print("Done! Total rows:", len(air_quality))


Fetched 1000 rows (offset=0)
Fetched 1000 rows (offset=1000)
Fetched 1000 rows (offset=2000)
Fetched 1000 rows (offset=3000)
Fetched 1000 rows (offset=4000)
Fetched 1000 rows (offset=5000)
Fetched 1000 rows (offset=6000)
Fetched 1000 rows (offset=7000)
Fetched 1000 rows (offset=8000)
Fetched 1000 rows (offset=9000)
Fetched 1000 rows (offset=10000)
Fetched 1000 rows (offset=11000)
Fetched 1000 rows (offset=12000)
Fetched 1000 rows (offset=13000)
Fetched 1000 rows (offset=14000)
Fetched 1000 rows (offset=15000)
Fetched 1000 rows (offset=16000)
Fetched 1000 rows (offset=17000)
Fetched 862 rows (offset=18000)
Done! Total rows: 18862


In [7]:
air_quality.head()

,unique_id,indicator_id,name,measure,measure_info,geo_type_name,geo_join_id,geo_place_name,time_period,start_date,data_value
0,878149,386,Ozone (O3),Mean,ppb,CD,210,Throgs Neck and Co-op City (CD10),Summer 2023,2023-06-01T00:00:00.000,35.69245438
1,876930,375,Nitrogen dioxide (NO2),Mean,ppb,UHF42,306,Chelsea - Clinton,Summer 2023,2023-06-01T00:00:00.000,21.23435622
2,874574,365,Fine particles (PM 2.5),Mean,mcg/m3,UHF34,402,West Queens,Summer 2023,2023-06-01T00:00:00.000,9.23668133
3,877212,375,Nitrogen dioxide (NO2),Mean,ppb,CD,317,East Flatbush (CD17),Summer 2023,2023-06-01T00:00:00.000,12.6709056
4,874912,365,Fine particles (PM 2.5),Mean,mcg/m3,CD,309,South Crown Heights and Lefferts Gardens (CD9),Summer 2023,2023-06-01T00:00:00.000,8.65103668


In [9]:
air_quality['time_period'].value_counts()

time_period
2015-2017              480
2005-2007              480
2009-2011              480
2012-2014              480
2017-2019              475
Summer 2022            423
Summer 2021            423
Summer 2016            423
Summer 2017            423
Summer 2015            423
Summer 2014            423
Summer 2012            423
Summer 2013            423
Summer 2019            423
Summer 2009            423
Summer 2011            423
Summer 2010            423
Summer 2020            423
Summer 2018            423
Summer 2023            423
2005                   417
2019                   321
2010                   321
Annual Average 2017    287
Winter 2020-21         282
Annual Average 2016    282
Winter 2016-17         282
Winter 2017-18         282
Annual Average 2018    282
Winter 2019-20         282
Annual Average 2020    282
Annual Average 2019    282
Winter 2018-19         282
Annual Average 2022    282
Winter 2021-22         282
Winter 2022-23         282
Annual Average 2

In [12]:
# Filter for rows with time_period containing '2023'
air_quality_2023 = air_quality[air_quality['time_period'].str.contains('Annual Average 2023', na=False)]

print(f"Filtered data shape: {air_quality_2023.shape}")
air_quality_2023.head()

Filtered data shape: (282, 11)


,unique_id,indicator_id,name,measure,measure_info,geo_type_name,geo_join_id,geo_place_name,time_period,start_date,data_value
423,874480,365,Fine particles (PM 2.5),Mean,mcg/m3,UHF42,305,Upper East Side,Annual Average 2023,2023-01-01T00:00:00.000,6.983049971
424,874860,365,Fine particles (PM 2.5),Mean,mcg/m3,CD,414,Rockaway and Broad Channel (CD14),Annual Average 2023,2023-01-01T00:00:00.000,6.124139701
425,874393,365,Fine particles (PM 2.5),Mean,mcg/m3,UHF34,206,Borough Park,Annual Average 2023,2023-01-01T00:00:00.000,6.58301364
426,874420,365,Fine particles (PM 2.5),Mean,mcg/m3,UHF34,101,Kingsbridge - Riverdale,Annual Average 2023,2023-01-01T00:00:00.000,6.800834953
427,876911,375,Nitrogen dioxide (NO2),Mean,ppb,UHF42,211,Williamsburg - Bushwick,Annual Average 2023,2023-01-01T00:00:00.000,18.15983736


In [31]:
# Create a pivot table grouping by geo_place_name and name, with data_value as values
air_quality = air_quality_2023.pivot_table(
    index='geo_place_name',
    columns='name',
    values='data_value', 
    aggfunc='first'
).reset_index()

air_quality.head()

name,geo_place_name,Fine particles (PM 2.5),Nitrogen dioxide (NO2)
0,Bay Ridge and Dyker Heights (CD10),6.603554291,17.71452907
1,Bayside - Little Neck,6.483274289,14.92994837
2,Bayside Little Neck-Fresh Meadows,6.474389448,15.04427297
3,Bayside and Little Neck (CD11),6.481628694,14.91951698
4,Bedford Stuyvesant (CD3),6.948465531,17.04140309


In [36]:
# Send to db:
engine = create_engine(f"postgresql+psycopg2://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}")

In [38]:
# Send df to PostgreSQL
air_quality.to_sql('air_quality', engine, if_exists='replace', index=False)


114